<a href="https://colab.research.google.com/github/himanshu-nakrani/TMLC-Gen-ai-projects/blob/main/openai_fimetuning_TMLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment**

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import json
from openai import OpenAI

In [ ]:
import pandas as pd

df = pd.read_json("hf://datasets/MakTek/Customer_support_faqs_dataset/train_expanded.json", lines=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# df = pd.read_csv('your_data.csv')  # Replace 'your_data.csv' with your dataset file

# Convert the dataset to JSONL format
output_file = 'data.jsonl'
with open(output_file, 'w') as f:
    for _, row in df.iterrows():
        # Create JSON lines for chat model fine-tuning
        json_line = json.dumps({
            "messages": [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": row['question']},
                {"role": "assistant", "content": row['answer']}
            ]
        })
        f.write(json_line + '\n')
print(f"Dataset converted and saved to {output_file}")

# comment when using your own data
# output_file = 'training.jsonl'

Dataset converted and saved to data.jsonl


In [ ]:
client = OpenAI()

In [ ]:
uploaded_file = client.files.create(
    file=open(output_file, "rb"),
    purpose="fine-tune"
)
print(f"File uploaded successfully. File ID: {uploaded_file.id}")

File uploaded successfully. File ID: file-K25b9hLM2VVUr1BfcoYDZ1


In [ ]:
fine_tune_job = client.fine_tuning.jobs.create(
    training_file=uploaded_file.id,
    suffix="custom-fine-tuned-model",
    model="gpt-4o-mini-2024-07-18"  # Adjust the model as required
)
print(f"Fine-tuning job started. Job ID: {fine_tune_job.id}")

Fine-tuning job started. Job ID: ftjob-JXQCwuDMblTEXjKh2ITcqDdy


In [ ]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-JXQCwuDMblTEXjKh2ITcqDdy', created_at=1735066236, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-8WCqOSfih3EAurulFOK4TlX6', result_files=[], seed=648803592, status='validating_files', trained_tokens=None, training_file='file-K25b9hLM2VVUr1BfcoYDZ1', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='custom-fine-tuned-model', method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 1.8}}}), FineTuningJob(id='ftjob-gyuWdvicX1D8V0JxNmZR6Geb', created_at=1735039222, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AhxKK2o9', finished_at=1735039518, hyperparameters=Hyper

In [ ]:
fine_tune_job

FineTuningJob(id='ftjob-JXQCwuDMblTEXjKh2ITcqDdy', created_at=1735066236, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-8WCqOSfih3EAurulFOK4TlX6', result_files=[], seed=648803592, status='validating_files', trained_tokens=None, training_file='file-K25b9hLM2VVUr1BfcoYDZ1', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='custom-fine-tuned-model', method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [ ]:
# Example call to the fine-tuned model
completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::AhxKK2o9",
    # Replace with the actual model name retrieved in above cell fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:custom-fine-tuned-model:AbOWr1n9'
    messages=[
        {"role": "system", "content": "You are a helpful assistant which acts as FAQ Support Assistant"},
        {"role": "user", "content": "what are all accepted payment methods?"}
    ]
)
print("Fine-tuned model response:", completion.choices[0].message.content)

Fine-tuned model response: We accept all major credit cards, debit cards, and PayPal.
